# OutputTrain

In [1]:
import pandas as pd
outputTrain = pd.read_csv('OutputTableTrain.csv')
outputTrain.head()

,ProblemID,UserID,SubmissionNumber,TimeStamp,TimeSinceLast,Grade,GradeDiff,NVideoEvents,NForumEvents,NumberOfThreadsLaunched,...,DistinctPostIds,NumberOfDownloads,DurationOfVideoActivity,NumberOfSpeedChange,EngagementIndex,NumberOfPosts,NumberOfLoads,SelectiveNumOfEvents,NVideoAndForum_,PlaysDownlsPerVideo
0,0,67b31325660be932fab1b1af1eddafc6,0,1392978845,NaN,66.666667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,5b32d46f935589e9518ebf35103d25a1,0,1394992883,NaN,100.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,d488b936db4019eebb541375aefcf8e2,0,1393252254,NaN,100.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,dc3de627b719e8e3491d0d6d8b19356b,0,1393513510,NaN,94.444497,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,41dce14eea3c94de39daeaba61698739,0,1394049763,NaN,83.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
outputTrain.columns

Index([u'ProblemID', u'UserID', u'SubmissionNumber', u'TimeStamp',
       u'TimeSinceLast', u'Grade', u'GradeDiff', u'NVideoEvents',
       u'NForumEvents', u'NumberOfThreadsLaunched', u'SeenVideo',
       u'ComAndPost', u'NumberOfPauses', u'NumberOfThreadViews',
       u'ScoreRelevantEvents', u'AverageVideoTimeDiffs', u'NumberOfPlays',
       u'NumberOfComments', u'DistinctIds', u'TotalVideoEvents',
       u'TotalForumEvents', u'DistinctPostIds', u'NumberOfDownloads',
       u'DurationOfVideoActivity', u'NumberOfSpeedChange', u'EngagementIndex',
       u'NumberOfPosts', u'NumberOfLoads', u'SelectiveNumOfEvents',
       u'NVideoAndForum_', u'PlaysDownlsPerVideo'],
      dtype='object')

In [3]:
len(outputTrain)

42954

In [4]:
from __future__ import division
outputTrain = outputTrain[~outputTrain['GradeDiff'].isnull()]
#/len(outputTrain)

In [5]:
outputTrain['NVideoAndForum']= outputTrain['NVideoEvents']+outputTrain['NForumEvents']
outputTrain = outputTrain[outputTrain['NVideoAndForum']>0]
#outputTrain= outputTrain[outputTrain['SubmissionNumber']>0]

In [6]:
outputTrain[outputTrain['SubmissionNumber']==0]

,ProblemID,UserID,SubmissionNumber,TimeStamp,TimeSinceLast,Grade,GradeDiff,NVideoEvents,NForumEvents,NumberOfThreadsLaunched,...,NumberOfDownloads,DurationOfVideoActivity,NumberOfSpeedChange,EngagementIndex,NumberOfPosts,NumberOfLoads,SelectiveNumOfEvents,NVideoAndForum_,PlaysDownlsPerVideo,NVideoAndForum


In [7]:
len(outputTrain)/42954

0.1151929971597523

In [8]:
improved_list = []
for i in range(0,len(outputTrain)):
    if outputTrain.iloc[i]["GradeDiff"]>0:
        improved_list.append(1)
    else:
        improved_list.append(0)
outputTrain['Improved'] = improved_list

In [9]:
# 0    1 
#3246 1702 
len(outputTrain[outputTrain['Improved']==0])/len(outputTrain)

0.6560226354082458

## New Features

In [ ]:
outputTrain['VideoThreads']=outputTrain['NVideoEvents']*outputTrain['NumberOfThreadViews']
outputTrain['SeenVideoForum']=outputTrain['SeenVideo']*outputTrain['NForumEvents']
outputTrain['VideoForumThreads'] = outputTrain['NVideoEvents']+outputTrain['NForumEvents']+outputTrain['NumberOfThreadViews']
    

time_per_video = []
for i in range(0,len(outputTrain)):
    if outputTrain.iloc[i]['NVideoEvents'] !=0:
        time_per_video.append(outputTrain.iloc[i]['DurationOfVideoActivity']/outputTrain.iloc[i]['NVideoEvents'])
    else:
        time_per_video.append(0)
outputTrain['TimeSpentPerVideo'] = time_per_video


# AGGREGATION OF DIFFERENCE BETWEEN VIDEOS CHECKED
aggregated_distinct_videos_change = []
problemIDs = []
userIDs = []
submissionNumber = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        difference = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)&(outputTrain['SubmissionNumber']==submission)]['DistinctIds']
            for distinctID in distinctIDs:
                aggregation = distinctID - difference
                aggregated_distinct_videos_change.append((aggregation))
                problemIDs.append(problem)
                userIDs.append(user)
                submissionNumber.append(submission)
                difference = distinctID
                
new_df = pd.DataFrame()
new_df['ProblemID'] = problemIDs
new_df['UserID'] = userIDs
new_df['SubmissionNumber'] = submissionNumber
new_df['DistinctVideoChange'] = aggregated_distinct_videos_change
outputTrain = new_df.merge(outputTrain,on=['ProblemID','UserID','SubmissionNumber'])
new_df.head()

#### MAXIMUM TIMESTAMP
maximum_timestamp = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        previous = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)&(outputTrain['SubmissionNumber']==submission)]['NVideoEvents']
            accumulation = distinctIDs + previous
            aggregated_distinct_videos_change.append((problem,user,submission,aggregation))
            difference = distinctIDs
            

# TOTAL NUMBER OF SUBMISSIONS
total_number_submissions = []
maximum_submission = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        count = 0
        maxi = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            count = count + 1
            maxi = submission
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            total_number_submissions.append(count)
            maximum_submission.append(maxi)
outputTrain['TotalNumberSubmissions'] = total_number_submissions
outputTrain['MaximumNumberSubmissions'] = maximum_submission 

# AVERAGE NUMBER OF PAUSES PER VIDEO:
outputTrain['AveragePausesPerVideo'] = outputTrain['NumberOfPauses']/outputTrain['DistinctIds']
# AVERAGE NUMBER OF PLAYS PER VIDEO:
outputTrain['AveragePlaysPerVideo'] = outputTrain['NumberOfPlays'] /outputTrain['DistinctIds']

outputTrain['AggregatingVideoForum'] = outputTrain['NumberOfPlays'] * outputTrain['NumberOfPosts']
outputTrain['PlaysTimesThreadViews'] = outputTrain['NumberOfPlays'] * outputTrain['NumberOfThreadViews']


from scipy.stats import pearsonr  
outputTrain['AverageNVideoEvents'] = outputTrain['NVideoEvents'] / outputTrain['DistinctIds']
outputTrain.fillna(0, inplace=True)
outputTrain['AverageNForumEvents'] = outputTrain['NForumEvents'] / outputTrain['DistinctPostIds']
outputTrain.fillna(0, inplace=True)
outputTrain['SumAverages'] = outputTrain['AverageNVideoEvents'] + outputTrain['AverageNForumEvents']


outputTrain['Speed'] = outputTrain['TimeSinceLast']/(1 + outputTrain['TotalNumberSubmissions'])

outputTrain['SubmissionsLeft'] = outputTrain['MaximumNumberSubmissions'] - outputTrain['SubmissionNumber']



In [10]:
outputTrain['VideoThreads']=outputTrain['NVideoEvents']*outputTrain['NumberOfThreadViews']
outputTrain['SeenVideoForum']=outputTrain['SeenVideo']*outputTrain['NForumEvents']
outputTrain['VideoForumThreads'] = outputTrain['NVideoEvents']+outputTrain['NForumEvents']+outputTrain['NumberOfThreadViews']
    

time_per_video = []
for i in range(0,len(outputTrain)):
    if outputTrain.iloc[i]['NVideoEvents'] !=0:
        time_per_video.append(outputTrain.iloc[i]['DurationOfVideoActivity']/outputTrain.iloc[i]['NVideoEvents'])
    else:
        time_per_video.append(0)
outputTrain['TimeSpentPerVideo'] = time_per_video

In [11]:
# AGGREGATION OF DIFFERENCE BETWEEN VIDEOS CHECKED
aggregated_distinct_videos_change = []
problemIDs = []
userIDs = []
submissionNumber = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        difference = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)&(outputTrain['SubmissionNumber']==submission)]['DistinctIds']
            for distinctID in distinctIDs:
                aggregation = distinctID - difference
                aggregated_distinct_videos_change.append((aggregation))
                problemIDs.append(problem)
                userIDs.append(user)
                submissionNumber.append(submission)
                difference = distinctID

In [12]:
new_df = pd.DataFrame()
new_df['ProblemID'] = problemIDs
new_df['UserID'] = userIDs
new_df['SubmissionNumber'] = submissionNumber
new_df['DistinctVideoChange'] = aggregated_distinct_videos_change
outputTrain = new_df.merge(outputTrain,on=['ProblemID','UserID','SubmissionNumber'])
new_df.head()

,ProblemID,UserID,SubmissionNumber,DistinctVideoChange
0,0,7e5b31a48d714c6a40a2a05e6824b819,1,1.0
1,0,7e5b31a48d714c6a40a2a05e6824b819,2,0.0
2,0,7e5b31a48d714c6a40a2a05e6824b819,4,0.0
3,0,7e5b31a48d714c6a40a2a05e6824b819,5,1.0
4,0,7e5b31a48d714c6a40a2a05e6824b819,6,-1.0


In [13]:
#### MAXIMUM TIMESTAMP
maximum_timestamp = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        previous = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)&(outputTrain['SubmissionNumber']==submission)]['NVideoEvents']
            accumulation = distinctIDs + previous
            aggregated_distinct_videos_change.append((problem,user,submission,aggregation))
            difference = distinctIDs

In [14]:
# TOTAL NUMBER OF SUBMISSIONS
total_number_submissions = []
maximum_submission = []
for problem in list(set(outputTrain['ProblemID'])):
    for user in list(set(outputTrain['UserID'])):
        count = 0
        maxi = 0
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            count = count + 1
            maxi = submission
        for submission in outputTrain[(outputTrain['ProblemID']==problem) & (outputTrain['UserID']==user)]['SubmissionNumber']:
            total_number_submissions.append(count)
            maximum_submission.append(maxi)
outputTrain['TotalNumberSubmissions'] = total_number_submissions
outputTrain['MaximumNumberSubmissions'] = maximum_submission 

In [15]:
# AVERAGE NUMBER OF PAUSES PER VIDEO:
outputTrain['AveragePausesPerVideo'] = outputTrain['NumberOfPauses']/outputTrain['DistinctIds']
# AVERAGE NUMBER OF PLAYS PER VIDEO:
outputTrain['AveragePlaysPerVideo'] = outputTrain['NumberOfPlays'] /outputTrain['DistinctIds']

In [16]:
outputTrain.fillna(0, inplace=True)

In [17]:
outputTrain.columns

Index([u'ProblemID', u'UserID', u'SubmissionNumber', u'DistinctVideoChange',
       u'TimeStamp', u'TimeSinceLast', u'Grade', u'GradeDiff', u'NVideoEvents',
       u'NForumEvents', u'NumberOfThreadsLaunched', u'SeenVideo',
       u'ComAndPost', u'NumberOfPauses', u'NumberOfThreadViews',
       u'ScoreRelevantEvents', u'AverageVideoTimeDiffs', u'NumberOfPlays',
       u'NumberOfComments', u'DistinctIds', u'TotalVideoEvents',
       u'TotalForumEvents', u'DistinctPostIds', u'NumberOfDownloads',
       u'DurationOfVideoActivity', u'NumberOfSpeedChange', u'EngagementIndex',
       u'NumberOfPosts', u'NumberOfLoads', u'SelectiveNumOfEvents',
       u'NVideoAndForum_', u'PlaysDownlsPerVideo', u'NVideoAndForum',
       u'Improved', u'VideoThreads', u'SeenVideoForum', u'VideoForumThreads',
       u'TimeSpentPerVideo', u'TotalNumberSubmissions',
       u'MaximumNumberSubmissions', u'AveragePausesPerVideo',
       u'AveragePlaysPerVideo'],
      dtype='object')

In [18]:
outputTrain['AggregatingVideoForum'] = outputTrain['NumberOfPlays'] * outputTrain['NumberOfPosts']
outputTrain['PlaysTimesThreadViews'] = outputTrain['NumberOfPlays'] * outputTrain['NumberOfThreadViews']

In [19]:
from scipy.stats import pearsonr  
outputTrain['AverageNVideoEvents'] = outputTrain['NVideoEvents'] / outputTrain['DistinctIds']
outputTrain.fillna(0, inplace=True)
outputTrain['AverageNForumEvents'] = outputTrain['NForumEvents'] / outputTrain['DistinctPostIds']
outputTrain.fillna(0, inplace=True)
outputTrain['SumAverages'] = outputTrain['AverageNVideoEvents'] + outputTrain['AverageNForumEvents']

a = outputTrain['SumAverages']
b = outputTrain['GradeDiff']
print 'SumAverages >>>>>>> '+ str(pearsonr(a,b))

SumAverages >>>>>>> (-0.067289947328159322, 2.1638689684879082e-06)


In [20]:
outputTrain['Speed'] = outputTrain['TimeSinceLast']/(1 + outputTrain['TotalNumberSubmissions'])

In [21]:
outputTrain['SubmissionsLeft'] = outputTrain['MaximumNumberSubmissions'] - outputTrain['SubmissionNumber']

In [222]:
outputTrain[['ProblemID','UserID','SubmissionNumber','DistinctVideoChange','TimeStamp','TimeSinceLast','GradeDiff','TotalNumberSubmissions','Speed','MaximumNumberSubmissions','SubmissionsLeft']]

,ProblemID,UserID,SubmissionNumber,DistinctVideoChange,TimeStamp,TimeSinceLast,GradeDiff,TotalNumberSubmissions,Speed,MaximumNumberSubmissions,SubmissionsLeft
0,0,7e5b31a48d714c6a40a2a05e6824b819,1,1.0,1393183782,228.0,-33.333329,5,3.800000e+01,6,5
1,0,7e5b31a48d714c6a40a2a05e6824b819,2,0.0,1393184051,269.0,33.333329,5,4.483333e+01,6,4
2,0,7e5b31a48d714c6a40a2a05e6824b819,4,0.0,1393184262,96.0,0.000000,5,1.600000e+01,6,2
3,0,7e5b31a48d714c6a40a2a05e6824b819,5,1.0,1393184358,96.0,-11.111164,5,1.600000e+01,6,1
4,0,7e5b31a48d714c6a40a2a05e6824b819,6,-1.0,1393184483,125.0,16.666667,5,2.083333e+01,6,0
5,0,7e0655c0aebaa7c60804b14efa05e320,1,1.0,1395433206,17615.0,16.666667,1,8.807500e+03,1,0
6,0,a51b2b6b9c03f01035bc9aeeaa48d94f,2,0.0,1393690637,61640.0,-11.111164,3,1.541000e+04,4,2
7,0,a51b2b6b9c03f01035bc9aeeaa48d94f,3,0.0,1393840168,149531.0,11.111164,3,3.738275e+04,4,1
8,0,a51b2b6b9c03f01035bc9aeeaa48d94f,4,11.0,1394149733,309565.0,-16.666667,3,7.739125e+04,4,0
9,0,8ceec64e19e75f25637e32b85bf41e6d,5,11.0,1395077713,1127540.0,-44.444501,1,5.637700e+05,5,0


In [22]:
fs = ['NumberOfPlaysPosts',u'ProblemID', u'SubmissionNumber', u'TimeStamp',
       u'TimeSinceLast', u'NVideoEvents',
       u'NForumEvents', u'NumberOfPlays', u'NumberOfPosts',
       u'NumberOfComments', u'DistinctIds', u'NumberOfLoads', u'SeenVideo',
       u'TotalVideoEvents', u'TotalForumEvents', u'NumberOfThreadsLaunched',
       u'NumberOfDownloads', u'ScoreRelevantEvents', u'SelectiveNumOfEvents',
       u'NumberOfPauses', u'NumberOfThreadViews', u'DurationOfVideoActivity',
       u'NumberOfSpeedChange', u'NVideoAndForum_', u'EngagementIndex',
       u'ComAndPost', u'AverageVideoTimeDiffs', u'PlaysDownlsPerVideo',
       u'NVideoAndForum', u'Improved', u'VideoThreads', u'SeenVideoForum',
       u'VideoForumThreads', u'TimeSpentPerVideo', u'AveragePausesPerVideo',
       u'AveragePlaysPerVideo','TotalNumberSubmissions','Speed','MaximumNumberSubmissions','SubmissionsLeft','AggregatingVideoForum']
from scipy.stats import pearsonr  
for feature in fs:
    a = outputTrain[feature]
    b = outputTrain['GradeDiff']
    print feature +" >>>>>>> "+ str(pearsonr(a,b))

KeyError: 'NumberOfPlaysPosts'

In [98]:
outputTrain[train_fs]

,Speed,TimeSinceLast,DistinctIds,VideoForumThreads,DurationOfVideoActivity,SelectiveNumOfEvents
0,3.800000e+01,228.0,1.0,0.0,0.0,1.0
1,4.483333e+01,269.0,1.0,0.0,233.0,4.0
2,1.600000e+01,96.0,1.0,0.0,0.0,1.0
3,1.600000e+01,96.0,2.0,0.0,43.0,4.0
4,2.083333e+01,125.0,1.0,0.0,0.0,0.0
5,8.807500e+03,17615.0,1.0,0.0,2982.0,8.0
6,1.541000e+04,61640.0,0.0,2.0,0.0,0.0
7,3.738275e+04,149531.0,0.0,4.0,0.0,0.0
8,7.739125e+04,309565.0,11.0,89.0,11397.0,6.0
9,5.637700e+05,1127540.0,11.0,0.0,1100429.0,134.0


# Normalizing the features

In [91]:
selective_fs = ['Speed','TimeSinceLast','DistinctIds','VideoForumThreads','DurationOfVideoActivity','SelectiveNumOfEvents','Improved']
X = outputTrain[train_fs]
train_fs = ['Speed','TimeSinceLast','DistinctIds','VideoForumThreads','DurationOfVideoActivity','SelectiveNumOfEvents']

In [140]:
outputTrain.to_csv('AggregatedFeatures.csv')

In [94]:
from sklearn import preprocessing
import numpy as np
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
scaled_df =pd.DataFrame(X_scaled,columns=train_fs) 
scaled_df['Improved'] = outputTrain['Improved']
scaled_df.head()

,Speed,TimeSinceLast,DistinctIds,VideoForumThreads,DurationOfVideoActivity,SelectiveNumOfEvents,Improved
0,-0.204439,-0.251125,0.164569,-0.311322,-0.117433,-0.114203,0
1,-0.204386,-0.250998,0.164569,-0.311322,-0.116351,-0.030285,1
2,-0.204610,-0.251531,0.164569,-0.311322,-0.117433,-0.114203,0
3,-0.204610,-0.251531,0.615696,-0.311322,-0.117233,-0.030285,0
4,-0.204573,-0.251442,0.164569,-0.311322,-0.117433,-0.142175,1


In [24]:
outputTrain.to_csv('TrainAllFeatures.csv')

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
neigh = KNeighborsClassifier(n_neighbors=5)
X_knn = scaled_df
y_knn = outputTrain['Improved']
X_train_knn, X_test_knn, y_train_binary_knn, y_test_binary_knn = train_test_split(X_knn,y_knn, test_size=0.1)
neigh.fit(X_train_knn, y_train_binary_knn) 
y_pred_train = neigh.predict(X_train_knn)
y_pred_test = neigh.predict(X_test_knn)
print('Accuracy Score: ',metrics.accuracy_score(y_test_binary_knn, y_pred_test))
print('Precision Score: ',metrics.precision_score(y_test_binary_knn, y_pred_test))
print('Recall Score: ',metrics.recall_score(y_test_binary_knn, y_pred_test))
print('f1_score: ',metrics.f1_score(y_test_binary_knn, y_pred_test))
metrics.roc_auc_score(y_test_binary_knn, y_pred_test)

('Accuracy Score: ', 0.60808080808080811)
('Precision Score: ', 0.43103448275862066)
('Recall Score: ', 0.2808988764044944)
('f1_score: ', 0.3401360544217687)


0.53634849183000743

In [175]:
X

array([[ -2.92259151e+06,   1.00487764e+04,  -8.18277853e+03,
         -4.80648034e+02,  -8.21383946e+01,  -1.79140192e+01],
       [ -2.92231843e+06,   1.01979732e+04,  -8.36305117e+03,
         -3.91769451e+02,  -6.37362279e+01,  -8.56528930e+00],
       [ -2.92211563e+06,   9.92102419e+03,  -8.25380675e+03,
         -4.80537094e+02,  -8.20134701e+01,  -1.79040070e+01],
       ..., 
       [  3.09886729e+06,  -1.81143568e+05,  -3.45239402e+04,
          1.42316253e+02,   3.00099246e+01,   5.49330958e+00],
       [  4.29140154e+05,  -9.55943099e+04,  -2.23673407e+04,
         -1.34357313e+02,  -2.04031112e+01,  -4.64904008e+00],
       [  5.09278977e+05,  -3.27765880e+04,   1.72892664e+04,
         -1.61938014e+02,  -8.61483686e+01,  -9.44297608e+00]])

## Random Forest

## PCA Decomposition

In [188]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
X = outputTrain[fs]
pca.fit(X)
print(pca.explained_variance_ratio_ )
X = pca.transform(X) 

[  9.32750621e-01   6.20376058e-02   5.18476863e-03   2.01920263e-05
   6.77917883e-06]


In [114]:
outputTrain['DifferenceDistinctVideos'] = aggregated_distinct_videos_change
outputTrain[['ProblemID','UserID','SubmissionNumber','AveragePausesPerVideo','AveragePlaysPerVideo','NForumEvents','Improved']]

,ProblemID,UserID,SubmissionNumber,AveragePausesPerVideo,AveragePlaysPerVideo,NForumEvents,Improved
37,0,2988b2553fc9985f66deed38c5f0a9b2,1,0.000000,0.000000,0.0,0
38,0,2988b2553fc9985f66deed38c5f0a9b2,2,0.000000,0.000000,0.0,1
60,0,7e0655c0aebaa7c60804b14efa05e320,1,4.000000,4.000000,0.0,1
62,0,c1ef777e51a07df0692e3fd3860d1c67,1,0.000000,1.000000,0.0,1
114,0,a51b2b6b9c03f01035bc9aeeaa48d94f,2,0.000000,0.000000,2.0,0
115,0,a51b2b6b9c03f01035bc9aeeaa48d94f,3,0.000000,0.000000,3.0,1
116,0,a51b2b6b9c03f01035bc9aeeaa48d94f,4,0.090909,0.090909,55.0,0
162,0,8e2f5d4779fd0af4d50dde9b1ac2fabb,1,0.000000,0.000000,17.0,1
175,0,034a58807553a3a5a742987370350bae,2,0.000000,0.000000,3.0,0
215,0,867505a5d6008d4ca5623e68a7b3ce46,1,4.000000,5.000000,0.0,1


In [158]:
outputTrain.isnull().any()

ProblemID                   False
UserID                      False
SubmissionNumber            False
DistinctVideoChange         False
TimeStamp                   False
TimeSinceLast               False
Grade                       False
GradeDiff                   False
NVideoEvents                False
NForumEvents                False
NumberOfPlays               False
NumberOfPosts               False
NumberOfComments            False
DistinctIds                 False
NumberOfLoads               False
SeenVideo                   False
TotalVideoEvents            False
TotalForumEvents            False
NumberOfThreadsLaunched     False
NumberOfDownloads           False
ScoreRelevantEvents         False
SelectiveNumOfEvents        False
NumberOfPauses              False
NumberOfThreadViews         False
DurationOfVideoActivity     False
NumberOfSpeedChange         False
NVideoAndForum_             False
EngagementIndex             False
ComAndPost                  False
AverageVideoTi

In [52]:
len(outputTrain[outputTrain['Improved']==1])

1702

In [50]:
#u'NumberOfPlays', u'NumberOfPosts',u'NumberOfComments', u'DistinctIds', u'NumberOfLoads', u'SeenVideo',u'TotalVideoEvents', u'TotalForumEvents', u'NumberOfThreadsLaunched',
#u'NumberOfDownloads', u'ScoreRelevantEvents', u'SelectiveNumOfEvents',u'NumberOfPauses', u'NumberOfThreadViews', u'DurationOfVideoActivity',
#u'NumberOfSpeedChange', u'NVideoAndForum_', u'EngagementIndex',
#u'ComAndPost', u'AverageVideoTimeDiffs', u'PlaysDownlsPerVideo'
outputTrain[(outputTrain['UserID']=='0123f0d5ea6362ba71ac18b22583d71d')][['SubmissionNumber','ProblemID','ScoreRelevantEvents','Improved']]#,'VideoThreads','SeenVideoForum','VideoForumThreads','TimeSpentPerVideo','Improved']]

,SubmissionNumber,ProblemID,ScoreRelevantEvents,Improved
7517,2,2,4.0,0
7518,3,2,7.0,0
7519,4,2,NaN,0
7520,5,2,1.0,0
7522,7,2,2.0,0
7523,8,2,2.0,0
7524,9,2,6.0,0
7525,10,2,2.0,1
13172,4,4,11.0,0
13175,7,4,1.0,1


# OutputTest

In [34]:
import pandas as pd
outputTest = pd.read_csv('OutputTableTest.csv')
outputTest.columns

Index([u'ProblemID', u'UserID', u'SubmissionNumber', u'TimeStamp',
       u'TimeSinceLast', u'Grade', u'GradeDiff', u'NVideoEvents',
       u'NForumEvents', u'NumberOfThreadsLaunched', u'SeenVideo',
       u'ComAndPost', u'NumberOfPauses', u'NumberOfThreadViews',
       u'ScoreRelevantEvents', u'AverageVideoTimeDiffs', u'NumberOfPlays',
       u'NumberOfComments', u'DistinctIds', u'TotalVideoEvents',
       u'TotalForumEvents', u'DistinctPostIds', u'NumberOfDownloads',
       u'DurationOfVideoActivity', u'NumberOfSpeedChange', u'EngagementIndex',
       u'NumberOfPosts', u'NumberOfLoads', u'SelectiveNumOfEvents',
       u'NVideoAndForum_', u'PlaysDownlsPerVideo'],
      dtype='object')

In [35]:
outputTest['NVideoAndForum']= outputTest['NVideoEvents']+outputTest['NForumEvents']
outputTest = outputTest[outputTest['NVideoAndForum']>0]

In [36]:
# TOTAL NUMBER OF SUBMISSIONS
total_number_submissions = []
maximum_submission = []
submissions = []
userIDs = []
problemIDs = []
size = 0
for problem in list(set(outputTest['ProblemID'])):
    for user in list(set(outputTest['UserID'])):
        count = 0
        maxi = 0
        for submission in outputTest[(outputTest['ProblemID']==problem) & (outputTest['UserID']==user)]['SubmissionNumber']:
            count = count + 1
            size = size + 1
            maxi = submission
            problemIDs.append(problem)
            userIDs.append(user)
            submissions.append(submission)
        for i in range(0,count):
            total_number_submissions.append(count)
            maximum_submission.append(maxi)


new_df = pd.DataFrame()
new_df['ProblemID'] = problemIDs
new_df['UserID'] = userIDs
new_df['SubmissionNumber'] = submissions
new_df['TotalNumberSubmissions'] = total_number_submissions
new_df['MaximumNumberSubmissions'] = maximum_submission 

outputTest = new_df.merge(outputTest,on=['ProblemID','UserID','SubmissionNumber'])
outputTest.head()

,ProblemID,UserID,SubmissionNumber,TotalNumberSubmissions,MaximumNumberSubmissions,TimeStamp,TimeSinceLast,Grade,GradeDiff,NVideoEvents,...,NumberOfDownloads,DurationOfVideoActivity,NumberOfSpeedChange,EngagementIndex,NumberOfPosts,NumberOfLoads,SelectiveNumOfEvents,NVideoAndForum_,PlaysDownlsPerVideo,NVideoAndForum
0,0,730a65f226b0663226b38ae00ccab79a,1,1,1,1394570921,27286.0,NaN,NaN,6.0,...,0.0,56.0,3.0,0.0,0.0,1.0,6.0,9.0,1.0,9.0
1,0,7f7c4dadd4d846382b5db460a0974fe9,2,1,2,1398614696,3813229.0,NaN,NaN,323.0,...,15.0,3560206.0,0.0,0.0,0.0,48.0,196.0,362.0,3.0,362.0
2,0,901c0195edc44042cf1ca29a73748440,1,1,1,1398692054,3996878.0,NaN,NaN,76.0,...,1.0,3975602.0,17.0,NaN,NaN,13.0,39.0,76.0,0.0,76.0
3,0,14ee173ca0c0006f5f6f049b633272f0,1,2,3,1393266677,91576.0,NaN,NaN,63.0,...,0.0,1939.0,0.0,NaN,NaN,5.0,39.0,63.0,3.0,63.0
4,0,14ee173ca0c0006f5f6f049b633272f0,3,2,3,1393356928,89956.0,NaN,NaN,7.0,...,0.0,74.0,0.0,NaN,NaN,1.0,5.0,7.0,2.0,7.0


In [37]:
outputTest['VideoForumThreads'] = outputTest['NVideoEvents']+outputTest['NForumEvents']+outputTest['NumberOfThreadViews']

In [38]:
# Speed
outputTest['Speed'] = outputTest['TimeSinceLast']/(1 + outputTest['TotalNumberSubmissions'])

In [39]:
outputTest['VideoThreads']=outputTest['NVideoEvents']*outputTest['NumberOfThreadViews']
outputTest['SeenVideoForum']=outputTest['SeenVideo']*outputTest['NForumEvents']
outputTest['VideoForumThreads'] = outputTest['NVideoEvents']+outputTest['NForumEvents']+outputTest['NumberOfThreadViews']
    

time_per_video = []
for i in range(0,len(outputTest)):
    if outputTest.iloc[i]['NVideoEvents'] !=0:
        time_per_video.append(outputTest.iloc[i]['DurationOfVideoActivity']/outputTest.iloc[i]['NVideoEvents'])
    else:
        time_per_video.append(0)
outputTest['TimeSpentPerVideo'] = time_per_video


# AGGREGATION OF DIFFERENCE BETWEEN VIDEOS CHECKED
aggregated_distinct_videos_change = []
problemIDs = []
userIDs = []
submissionNumber = []
for problem in list(set(outputTest['ProblemID'])):
    for user in list(set(outputTest['UserID'])):
        difference = 0
        for submission in outputTest[(outputTest['ProblemID']==problem) & (outputTest['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTest[(outputTest['ProblemID']==problem) & (outputTest['UserID']==user)&(outputTest['SubmissionNumber']==submission)]['DistinctIds']
            for distinctID in distinctIDs:
                aggregation = distinctID - difference
                aggregated_distinct_videos_change.append((aggregation))
                problemIDs.append(problem)
                userIDs.append(user)
                submissionNumber.append(submission)
                difference = distinctID
                
new_df = pd.DataFrame()
new_df['ProblemID'] = problemIDs
new_df['UserID'] = userIDs
new_df['SubmissionNumber'] = submissionNumber
new_df['DistinctVideoChange'] = aggregated_distinct_videos_change
outputTest = new_df.merge(outputTest,on=['ProblemID','UserID','SubmissionNumber'])

#### MAXIMUM TIMESTAMP
maximum_timestamp = []
for problem in list(set(outputTest['ProblemID'])):
    for user in list(set(outputTest['UserID'])):
        previous = 0
        for submission in outputTest[(outputTest['ProblemID']==problem) & (outputTest['UserID']==user)]['SubmissionNumber']:
            distinctIDs = outputTest[(outputTest['ProblemID']==problem) & (outputTest['UserID']==user)&(outputTest['SubmissionNumber']==submission)]['NVideoEvents']
            accumulation = distinctIDs + previous
            aggregated_distinct_videos_change.append((problem,user,submission,aggregation))
            difference = distinctIDs
            

# AVERAGE NUMBER OF PAUSES PER VIDEO:
outputTest['AveragePausesPerVideo'] = outputTest['NumberOfPauses']/outputTest['DistinctIds']
# AVERAGE NUMBER OF PLAYS PER VIDEO:
outputTest['AveragePlaysPerVideo'] = outputTest['NumberOfPlays'] /outputTest['DistinctIds']

outputTest['AggregatingVideoForum'] = outputTest['NumberOfPlays'] * outputTest['NumberOfPosts']
outputTest['PlaysTimesThreadViews'] = outputTest['NumberOfPlays'] * outputTest['NumberOfThreadViews']

outputTest['AverageNVideoEvents'] = outputTest['NVideoEvents'] / outputTest['DistinctIds']
outputTest.fillna(0, inplace=True)
outputTest['AverageNForumEvents'] = outputTest['NForumEvents'] / outputTest['DistinctPostIds']
outputTest.fillna(0, inplace=True)
outputTest['SumAverages'] = outputTest['AverageNVideoEvents'] + outputTest['AverageNForumEvents']


outputTest['Speed'] = outputTest['TimeSinceLast']/(1 + outputTest['TotalNumberSubmissions'])

outputTest['SubmissionsLeft'] = outputTest['MaximumNumberSubmissions'] - outputTest['SubmissionNumber']



In [52]:
outputTrain.shape

(4948, 49)

In [51]:
outputTest.shape

(2772, 48)

In [ ]:
outputTest.to_csv('TestAllFeatures.csv')

In [145]:
fs_test = ['Speed','TimeSinceLast','DistinctIds','VideoForumThreads','DurationOfVideoActivity','SelectiveNumOfEvents','ProblemID','UserID','SubmissionNumber']
outputTest.fillna(0, inplace=True)
X_test = outputTest[fs_test]
X_test.to_csv('outpuTestAggregated.csv')
len(X_test)

2772

## Loading Pickle File

In [67]:
#raw_pickle = pd.read_pickle("dataset.pickle")
raw_pickle[0]['2988b2553fc9985f66deed38c5f0a9b2']

{0: {'Grade': 77.77783075968424, 'TimeStamp': 1393879520},
 1: {'Features': {},
  'Grade': 77.77783075968424,
  'GradeDiff': 0.0,
  'NForumEvents': 0,
  'NVideoEvents': 8,
  'TimeSinceLast': 681844,
  'TimeStamp': 1394561364,
  'VideoEvents': {'CurrentTime': [None,
    None,
    None,
    None,
    None,
    None,
    None,
    None],
   'EventType': ['Video.Load',
    'Video.Load',
    'Video.Load',
    'Video.Load',
    'Video.Load',
    'Video.Load',
    'Video.Load',
    'Video.Load'],
   'NewSpeed': [None, None, None, None, None, None, None, None],
   'NewTime': [None, None, None, None, None, None, None, None],
   'OldSpeed': [None, None, None, None, None, None, None, None],
   'OldTime': [None, None, None, None, None, None, None, None],
   'SeekType': [None, None, None, None, None, None, None, None],
   'TimeStamp': [1393880648,
    1393886354,
    1394355507,
    1394398146,
    1394398961,
    1394399715,
    1394399719,
    1394399722],
   'VideoID': ['25', '11', '27', '11', '